In [1]:
using ExpFamilyPCA
using CompressedBeliefMDPs

using BenchmarkTools
using Random
using Plots
using Statistics
using Distances

Random.seed!(1)

TaskLocalRNG()

In [2]:
n_corridors = 2
corridor_length = 50
maze = CircularMaze(n_corridors, corridor_length)
sampler = PolicySampler(maze, n=200)
raw_beliefs = sampler(maze)
beliefs = make_numerical(raw_beliefs, maze)

92×101 Matrix{Float64}:
 0.00626732  0.00624271  0.00626732  …  0.00646361  0.00634106  0.0
 0.0         0.0         0.0            0.0         0.0         1.0
 0.0124854   0.0125346   0.0126821      0.0         0.0         0.0
 0.0165572   0.0166623   0.0168181      0.0         0.0         0.0
 0.0184335   0.0184212   0.0184335      0.0         0.0         0.0
 0.0193284   0.019361    0.0194036   …  0.0         0.0         0.0
 0.0196873   0.0196848   0.0196873      0.0         0.0         0.0
 0.0198663   0.0198728   0.0198813      0.0         0.0         0.0
 0.0199378   0.0199373   0.0199378      0.0         0.0         0.0
 0.0199734   0.0199747   0.0199764      0.0         0.0         0.0
 ⋮                                   ⋱                          ⋮
 0.0         0.0         0.0            0.0198597   0.0198619   0.0
 0.0184335   0.0184212   0.0184335      0.0         0.0         0.0
 0.0192953   0.0193064   0.0193284   …  0.0         0.0         0.0
 0.0196873   0.0196848   0

In [5]:
outdim = 3

3

In [6]:
@benchmark ExpFamilyPCA.fit!(
	PoissonEPCA(size(beliefs)[2], outdim),
	beliefs;
	maxiter=30,
)

BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 15.507 s (7.22% GC) to evaluate,
 with a memory estimate of 23.53 GiB, over 1112649 allocations.

In [8]:
@benchmark ExpFamilyPCA.fit!(
	EPCA(
	    size(beliefs)[2],
	    outdim,
	    Distances.gkl_divergence,
	    exp,
	    Val((:Bregman, :g));
	    μ=1,
	    ϵ=eps()
	),
	beliefs;
	maxiter=30,
)

BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 15.742 s (8.21% GC) to evaluate,
 with a memory estimate of 23.53 GiB, over 1112648 allocations.